In [2]:
import os
import drjit
gpu_num = [0, 1, 2, 3, 4, 5] # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import sionna
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
tf.get_logger().setLevel('ERROR')
# tf.config.run_functions_eagerly(True)

from jammer.simulation_model import *
import jammer.simulation_model as sim
from tensorflow.python.keras.losses import MeanAbsoluteError, MeanSquaredError, BinaryCrossentropy

In [3]:
# common parameters
model_parameters = {}
jammer_parameters = {}
model_parameters["perfect_csi"] = False
model_parameters["num_ut"] = 1
model_parameters["jammer_present"] = True
model_parameters["jammer_power"] = 1.0
model_parameters["jammer_mitigation"] = "pos"
model_parameters["jammer_mitigation_dimensionality"] = 1
model_parameters["num_silent_pilot_symbols"] = 4
jammer_parameters["trainable"] = True
model_parameters["jammer_parameters"] = jammer_parameters

In [4]:
# needed, as keras MSE does not take |.|
abs_mse = lambda y_true, y_pred: tf.reduce_mean(tf.square(tf.abs(y_true - y_pred)))
abs_log = lambda y_true, y_pred: tf.reduce_mean(tf.math.log(tf.abs(y_true - y_pred) + 1))
# name, loss_fn, over symbols?, loss_over_logits
parameters = [
    ("L1 over symbols", negative_function(MeanAbsoluteError()), True, False),
    ("MSE over symbols", negative_function(abs_mse), True, False),
    ("L1 over bit estimates", negative_function(MeanAbsoluteError()), False, False),
    ("MSE over bit estimates", negative_function(MeanSquaredError()), False, False),
    ("BCE over bit estimates (logits)", BinaryCrossentropy(from_logits=True), False, True),
    ("log over bit estimates", negative_function(abs_log), False, False),
]
sim.BATCH_SIZE = 2

In [5]:
@tf.function
def train_fn(parameters):
    for name, loss_fn, over_symbols, loss_over_logits in parameters:
        model = Model(**model_parameters, return_symbols=over_symbols)
        train_model(model,
                    loss_fn=loss_fn,
                    loss_over_logits=loss_over_logits,
                    weights_filename=f"weights/{name}.pickle",
                    log_tensorboard=True,
                    log_weight_images=True,
                    show_final_weights=True)

train_fn(parameters)

OperatorNotAllowedInGraphError: in user code:

    File "/tmp/ipykernel_2398664/767751672.py", line 4, in train_fn  *
        model = Model(**model_parameters, return_symbols=over_symbols)
    File "/home/msc23h6/MSDocuments/master_thesis/code/mine/jammer_mitigation/jammer/simulation_model.py", line 141, in __init__  **
        pilot_pattern = OneHotPilotPattern(self._num_silent_pilot_symbols, self._num_tx, self._num_streams_per_tx, self._num_ofdm_symbols, self._fft_size)
    File "/home/msc23h6/MSDocuments/master_thesis/code/mine/jammer_mitigation/jammer/custom_pilots.py", line 115, in __init__
        super().__init__(mask, pilots, trainable=False, normalize=normalize,
    File "/scratch/fabianu/.conda/envs/sionna-env/lib/python3.9/site-packages/sionna/ofdm/pilot_pattern.py", line 50, in __init__
        self._check_settings()
    File "/scratch/fabianu/.conda/envs/sionna-env/lib/python3.9/site-packages/sionna/ofdm/pilot_pattern.py", line 121, in _check_settings
        assert tf.ran